In [2]:
from Prompts import prompter
from dotenv import load_dotenv
import os
from google import genai

load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

client = genai.Client()

In [4]:
import json

# Path to your JSON file
file_path = "Datasets/network_analysis_2/na2.json"

# Load the JSON content
with open(file_path, "r") as f:
    data = json.load(f)

# Extract parts from the JSON
scenario = data["scenario"]
tools = data["tools"]
questions = data["questions"]
file=data["files"]



In [ ]:
Answer_list=[]
for i in range(0, len(questions)):

    j = 0
    max_attempts = 4 #max per question
    history=[]  #here only used to store (thought, action, observation)
    
    prompt = prompter.prompt_manual_thought(i, scenario, tools, file, questions, history) #prompt that trigger thought
    
    response = client.models.generate_content(    # generates thought
        model="gemini-2.5-flash", contents=prompt
    )
    thought=response.text

    print(f"🧠 Thought generated for question {i+1}: {thought}")
    prompt=prompter.prompt_manual_action(i,scenario,tools, file,questions, history, thought) #prompt to generate action

    response = client.models.generate_content(    # generates command based on the thought
        model="gemini-2.5-flash", contents=prompt
    )
    action=response.text
    
    print(f"🛠️ Please execute this command for question {i+1}: {action}")
    
    while j < max_attempts:

        user_command_output = input("""Please paste the command's output here or type "0" to skip question :\n""")
        if user_command_output== "0":
            print(f"⛔ Skipped question {i+1}.")
            Answer_list.append("Skipped")
            break
        user_command_output = user_command_output[:5000]   # Limit to 500 characters(long outputs ruins agent's performance+ take long time)
        history.append({"previous Thought": [thought],"previous command": [action], "previous command output": [user_command_output]})
        print(f"📜 {history}")
        prompt = prompter.prompt_manual_thought(i, scenario, tools, file, questions, history)  # prompt to generate next thought
        
        response = client.models.generate_content(       # generates next thought based on the command output
            model="gemini-2.5-flash",
            contents=prompt
        )
        thought = response.text

        print(f"🧠 Thought generated for question {i+1}: {thought}")

        prompt=prompter.prompt_manual_action(i,scenario,tools, file,questions, history, thought) #prompt to generate action

        response = client.models.generate_content(    # generates command based on the thought
            model="gemini-2.5-flash", contents=prompt
        )
        action=response.text
        
        

        if "final answer:" in action.lower():
            text = action.strip()
            answer_part = text.lower().split("final answer:")[-1].strip()
            print(f"✅ Answer to the question {i+1}: {answer_part}")
            Answer_list.append(answer_part)
            break
        else:
            print(f"🛠️ Please execute this command for question {i+1}: {action}")            
        
        j += 1
    if j == max_attempts:
        Answer_list.append("Answer not found")  

print("✅ Task finished.")        


🧠 Thought generated for question 1: <internal reasoning>
The goal is to identify the IP responsible for port scanning. Port scanning involves one IP attempting to connect to multiple ports on another IP. I need to analyze the PCAP file for these patterns.

1.  **Filter for initial connection attempts:** Port scans are typically initiated using SYN packets. I should filter the traffic to only show TCP packets where the SYN flag is set and the ACK flag is not set (`tcp.flags.syn == 1 and tcp.flags.ack == 0`). This isolates the first packet of a TCP handshake.
2.  **Extract relevant fields:** For each such packet, I need to know the source IP (`ip.src`), the destination IP (`ip.dst`), and the destination port (`tcp.dstport`).
3.  **Count unique destination ports per source/destination IP pair:** To identify a scanner, I need to count how many *unique* destination ports a specific source IP attempts to connect to on a specific destination IP. An IP conducting a scan will have a high number

In [5]:
Answer_list=[]
for i in range(0, len(questions)):

    j = 0
    max_attempts = 4 #max per question
    history=[]  #here only used to store (thought, action, observation)
    
    prompt = prompter.prompt_manual_thought(i, scenario, tools, file, questions, history) #prompt that trigger thought
    
    response = client.models.generate_content(    # generates thought
        model="gemini-2.5-flash", contents=prompt
    )
    thought=response.text

    print(f"🧠 Thought generated for question {i+1}: {thought}")
    prompt=prompter.prompt_manual_action(i,scenario,tools, file,questions, history, thought) #prompt to generate action

    response = client.models.generate_content(    # generates command based on the thought
        model="gemini-2.5-flash", contents=prompt
    )
    action=response.text
    
    print(f"🛠️ Please execute this command for question {i+1}: {action}")
    
    while j < max_attempts:

        user_command_output = input("""Please paste the command's output here or type "0" to skip question :\n""")
        if user_command_output== "0":
            print(f"⛔ Skipped question {i+1}.")
            Answer_list.append("Skipped")
            break
        user_command_output = user_command_output[:5000]   # Limit to 500 characters(long outputs ruins agent's performance+ take long time)
        history.append({"previous Thought": [thought],"previous command": [action], "previous command output": [user_command_output]})
        print(f"📜 {history}")
        prompt = prompter.prompt_manual_thought(i, scenario, tools, file, questions, history)  # prompt to generate next thought
        
        response = client.models.generate_content(       # generates next thought based on the command output
            model="gemini-2.5-flash",
            contents=prompt
        )
        thought = response.text

        print(f"🧠 Thought generated for question {i+1}: {thought}")

        prompt=prompter.prompt_manual_action(i,scenario,tools, file,questions, history, thought) #prompt to generate action

        response = client.models.generate_content(    # generates command based on the thought
            model="gemini-2.5-flash", contents=prompt
        )
        action=response.text
        
        

        if "final answer:" in action.lower():
            text = action.strip()
            answer_part = text.lower().split("final answer:")[-1].strip()
            print(f"✅ Answer to the question {i+1}: {answer_part}")
            Answer_list.append(answer_part)
            break
        else:
            print(f"🛠️ Please execute this command for question {i+1}: {action}")            
        
        j += 1
    if j == max_attempts:
        Answer_list.append("Answer not found")  

print("✅ Task finished.")        


🧠 Thought generated for question 1: <internal reasoning>
The goal is to identify the IP responsible for port scanning. Port scanning involves one IP attempting to connect to multiple ports on another IP. I need to analyze the PCAP file for these patterns.

1.  **Filter for initial connection attempts:** Port scans are typically initiated using SYN packets. I should filter the traffic to only show TCP packets where the SYN flag is set and the ACK flag is not set (`tcp.flags.syn == 1 and tcp.flags.ack == 0`). This isolates the first packet of a TCP handshake.
2.  **Extract relevant fields:** For each such packet, I need to know the source IP (`ip.src`), the destination IP (`ip.dst`), and the destination port (`tcp.dstport`).
3.  **Count unique destination ports per source/destination IP pair:** To identify a scanner, I need to count how many *unique* destination ports a specific source IP attempts to connect to on a specific destination IP. An IP conducting a scan will have a high number

In [6]:
print(Answer_list)


['10.251.96.4', '1014-48994', 'tcp syn scan', 'Skipped', 'upload.php', 'Skipped', 'Skipped', 'Skipped', 'sh', '4422']


In [7]:
#saves all the answers to a file for evaluation
base_name = os.path.splitext(os.path.basename(file_path))[0]

output_file = os.path.join("Evaluation","manual","ReAct", f"{base_name}.txt")

with open(output_file, 'w') as f:
    for answer in Answer_list:
        f.write(answer + "\n")

print(f"Answers saved to {output_file}")

Answers saved to Evaluation/manual/ReAct/na2.txt
